In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import pickle


np.random.seed(42)
if np.random.choice(np.arange(1000)) != 102:
    raise ValueError("Random seed is not set correctly.")

In [2]:
# 1) Load and preprocess the Goodreads dataset
DATASET = 'goodreads'
base_artifacts = Path.cwd().resolve().parents[2] / 'CausalI2I_artifacts'
data = pd.read_csv(
    base_artifacts / 'Datasets' / 'Raw' / DATASET / f'goodreads_filtered.csv'
)
data.columns = ['user_id', 'item_id', 'timestamp']
data['interaction'] = 1
data = data.sort_values(by='timestamp')

In [3]:
# Drop duplicate user-item interactions, keeping only the first occurrence
ser = data[['user_id', 'item_id']].value_counts()
doubles_list = list(ser[ser > 1].index)
for u_id, i_id in doubles_list:
    x = data[(data['user_id'] == u_id) & (data['item_id'] == i_id)].index
    data = data.drop(x[1:])

In [4]:
# 2) Get unique users, items, and counts
unique_users = data['user_id'].unique()
unique_items = data['item_id'].unique()

n_users = len(unique_users)
n_items = len(unique_items)

print(f"Number of unique users: {n_users:,}")
print(f"Number of unique items: {n_items:,}")

Number of unique users: 7,801
Number of unique items: 6,384


In [5]:
# 3) Divide users and items into train and test sets
test_users = np.random.choice(unique_users, size=int(0.5 * n_users), replace=False)
train_users = np.setdiff1d(unique_users, test_users)
test_items = np.random.choice(unique_items, size=int(0.2 * n_items), replace=False)
train_items = np.setdiff1d(unique_items, test_items)

In [6]:
# 4) Create train and test sets
all_pairs = pd.merge(
    pd.DataFrame({'user_id': unique_users}), 
    pd.DataFrame({'item_id': unique_items}),
    how='cross'
)
data_imp = pd.merge(
    left=all_pairs, 
    right=data, 
    on=['user_id', 'item_id'], 
    how='left'
)
data_imp = data_imp[['user_id', 'item_id', 'interaction', 'timestamp']]
data_imp['interaction'] = data_imp['interaction'].fillna(0).astype(int)

train = data_imp[data_imp['user_id'].isin(train_users) | data_imp['item_id'].isin(train_items)].copy()
test =  data_imp[data_imp['user_id'].isin(test_users) & data_imp['item_id'].isin(test_items)].copy()
print("Proportion of 1's in imputed sets:")
print(f"    Train:  {train['interaction'].mean():.2%}")
print(f"    Test:   {test['interaction'].mean():.2%}")

Proportion of 1's in imputed sets:
    Train:  1.75%
    Test:   1.70%


In [7]:
# 5) Reindex users and items to start from 0
old2new_users = {old: new for new, old in enumerate(unique_users)}
old2new_items = {old: new for new, old in enumerate(unique_items)}

train['user_id'] = train['user_id'].map(old2new_users)
train['item_id'] = train['item_id'].map(old2new_items)
test['user_id'] = test['user_id'].map(old2new_users)
test['item_id'] = test['item_id'].map(old2new_items)

item_dict = {v: k for k, v in old2new_items.items()}

# Data for SASRec
data['user_id'] = data['user_id'].map(old2new_users)
data['item_id'] = data['item_id'].map(old2new_items)
data = data.sort_values(by=['user_id', 'timestamp', 'item_id']).reset_index(drop=True)

In [ ]:
# 6) Save to disk
print("Saving files to disk...")
train.to_csv(base_artifacts / 'Datasets' / 'Processed' / DATASET / 'train.csv', index=False)
test.to_csv(base_artifacts / 'Datasets' / 'Processed' / DATASET / 'test.csv', index=False)
data.to_csv(base_artifacts / 'Datasets' / 'Processed' / DATASET / 'data_sasrec.csv', index=False)

with open(base_artifacts / 'Datasets' / 'Processed' / DATASET / 'item_dict.pkl', 'wb') as f:
    pickle.dump(item_dict, f)

print('Files saved to disk.')

Saving files to disk...
Files saved to disk.


In [9]:
# 7) # Choose 10K pairs
full_data = pd.concat([train, test], axis=0)
big_pivot = full_data.pivot(index='user_id', columns='item_id', values='interaction')
test_items = test['item_id'].unique()

X = big_pivot[test_items].values
mean = X.mean(axis=0)
std  = np.maximum(X.std(axis=0, ddof=1), 1e-8)
M = (X - mean) / std
corr_mat = (M.T @ M) / (M.shape[0] - 1)
np.fill_diagonal(corr_mat, 0)

best_flat_pair_idx = corr_mat.flatten().argsort()[-10000:]
best_pairs_idx = [(i % len(test_items), i // len(test_items)) for i in best_flat_pair_idx]
best_pairs_titles = [(item_dict[test_items[i]], item_dict[test_items[j]]) for i, j in best_pairs_idx]

In [ ]:
with open(base_artifacts / 'Chosen_Pairs' / f'{DATASET}_chosen_pairs.pkl', "wb") as f:
    pickle.dump(best_pairs_titles, f)

In [ ]:
# with open(base_artifacts / 'Chosen_Pairs' / f'{DATASET}_chosen_pairs.pkl', "rb") as f:
#     up = pickle.load(f)